In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.cm import get_cmap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipywidgets import interact 

# this try-except block is for Google Colab, which does not install all requirements automatically
try:
    from colorspacious import cspace_converter
except ModuleNotFoundError:
    !pip install colorspacious
    from colorspacious import cspace_converter

np.random.seed(1234)

colormaps=["viridis", "jet", "gray", "PiYG", "flag", "Set1", "turbo", "hsv"]

img_width = 400
ls  = np.linspace(0, 10, img_width)
x,y = np.meshgrid(ls, ls)

noise_lvl = 0.3
data = np.sin(x) * np.cos(y)

def convert_cm_gray(cmname: str):
    x = np.linspace(0.0, 1.0, 100)
    rgb = get_cmap(plt.get_cmap(cmname))(x)[np.newaxis, :, :3]
    lab = cspace_converter("sRGB1", "CAM02-UCS")(rgb)
    L = lab[0, :, 0]
    L = np.float32(np.vstack((L, L, L))).transpose() / 100

    return ListedColormap(L)

    
@interact(colormap=colormaps, gray=False, noise=(0, 0.4, 0.05))
def plot_data(colormap=colormaps[0], gray=False, noise=noise_lvl):
    fig, ax = plt.subplots(figsize=(6,6))
    
    if gray:
        colormap = convert_cm_gray(colormap)
    
    im_data = data + noise * np.random.rand(img_width, img_width)
    
    im = ax.imshow(im_data, cmap=colormap)
    ax.invert_yaxis()
    ax.set_title("Some data")
    
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1)
    cbar = plt.colorbar(im, cax=cax)
    cbar.set_label('Value')
        
    plt.tight_layout()
    plt.show()

In [ ]:
@interact(noise=(0, 0.4, 0.05))
def plot_all_data(noise=0.0):

    im_data = data + noise * np.random.rand(img_width, img_width)
    
    fig, axs = plt.subplots(len(colormaps), 2,  figsize=(8, 20), frameon=False)

    for (index, cm_name) in enumerate(colormaps):

        # normal version
        im = axs[index, 0].imshow(im_data, cmap=cm_name)
        axs[index, 0].invert_yaxis()
        axs[index, 0].set_title(cm_name)

        divider = make_axes_locatable(axs[index, 0])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cbar = plt.colorbar(im, cax=cax)

        # gray scale version
        gray_cm = convert_cm_gray(cm_name)

        im_g = axs[index, 1].imshow(im_data, cmap=gray_cm)
        axs[index, 1].invert_yaxis()
        axs[index, 1].set_title(cm_name + " gray")

        divider = make_axes_locatable(axs[index, 1])
        cax = divider.append_axes("right", size="5%", pad=0.1)
        cbar = plt.colorbar(im_g, cax=cax)


    plt.tight_layout()
    plt.show()